In [1]:
import pandas as pd
import numpy as np
import datetime
import math as mat
import time

In [2]:
jobs_df = pd.read_csv('july-2017-thru-june-2018.txt', sep='|', nrows=1000000)

In [3]:
jobs_df.head()

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,State,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName
0,15806649,15806649,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1085,WOODWARD_TCP-x-228324-x-228324-x-301-x-Multi_A...
1,15806664,15806664,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1099,WOODWARD_TCP-x-115174-x-115174-x-301-x-Multi_A...
2,15806670,15806670,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1126,WOODWARD_TCP-x-215607-x-215607-x-301-x-Multi_A...
3,15806671,15806671,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp230,WOODWARD_TCP-x-116214-x-116214-x-1701-x-Multi_...
4,15811617,15811617,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01T05:00:00,...,COMPLETED,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_aa7c22e04f21


Are there any nodes that fail more than the other ones?

In [25]:
jobs_df.State.unique()

array(['COMPLETED', 'TIMEOUT', 'CANCELLED by 29607', 'FAILED',
       'CANCELLED by 454080', 'CANCELLED by 100684',
       'CANCELLED by 667840', 'CANCELLED by 660360',
       'CANCELLED by 481000', 'CANCELLED by 0', 'CANCELLED by 229215',
       'CANCELLED by 633620', 'CANCELLED by 207302',
       'CANCELLED by 124006', 'CANCELLED by 112602',
       'CANCELLED by 172079', 'CANCELLED by 199052',
       'CANCELLED by 156369', 'CANCELLED by 389801', 'NODE_FAIL',
       'CANCELLED by 123148', 'CANCELLED by 501421',
       'CANCELLED by 525961', 'CANCELLED by 352698',
       'CANCELLED by 169859', 'CANCELLED by 144605', 'CANCELLED by 85636',
       'CANCELLED by 240352', 'CANCELLED by 195638',
       'CANCELLED by 534784', 'CANCELLED by 379605',
       'CANCELLED by 308082', 'CANCELLED by 28075', 'CANCELLED by 337100',
       'CANCELLED by 497968', 'CANCELLED by 485800',
       'CANCELLED by 237813', 'CANCELLED by 504485',
       'CANCELLED by 174942', 'CANCELLED by 171742',
       'CANCEL

In [4]:
jobs_mask = jobs_df['NodeList'].str[3]!='['
jobs_mask.head()

0    True
1    True
2    True
3    True
4    True
Name: NodeList, dtype: bool

In [5]:
Jobs_df=jobs_df[jobs_mask]
Jobs_df.shape

(999112, 25)

In [127]:
#Slice the dataframe to show all failed jobs
jobs_failure = Jobs_df[(Jobs_df.State=="FAILED") | (Jobs_df.State=="NODE_FAIL")]
#jobs_failure = Jobs_df[Jobs_df.State=="NODE_FAIL"]
jobs_failure.State.unique()
# jobs_failure.shape

array(['FAILED', 'NODE_FAIL'], dtype=object)

In [128]:
#Drop unused columns
jobs_failed=jobs_failure[['JobID', 'ExitCode', 'State', 'NodeList']]
jobs_failed.head()

,JobID,ExitCode,State,NodeList
386,15812080,1:0,FAILED,vmp1106
441,15812174,1:0,FAILED,vmp1119
453,15812190,1:0,FAILED,vmp1122
537,15812298,1:0,FAILED,vmp1090
543,15812304,1:0,FAILED,vmp1091


In [103]:
#Count how many times each node has failed
pd.DataFrame(jobs_failed.NodeList.value_counts());

In [104]:
#Count how many times every node has been used overall
pd.DataFrame(Jobs_df.NodeList.value_counts());

In [131]:
failed = {}
for key, value in jobs_failed[jobs_failed.State.isin(['FAILED', 'NODE_FAIL'])].groupby('NodeList').size().iteritems():
    failed.update({key: value})  

In [132]:
failed_df = pd.DataFrame.from_dict(failed, orient='index', columns=["failures"])
# failed_df['node'] = failed_df.index
failed_df.reset_index(level=0, inplace=True)
# failed_df.columns(['node', 'failures'])
failed_df.head()

,index,failures
0,None assigned,45
1,gpu0001,141
2,gpu0002,142
3,gpu0003,100
4,gpu0004,105


In [133]:
total = {}
for key, value in Jobs_df.groupby('NodeList').size().iteritems():
    total.update({key: value})  

In [134]:
total_df = pd.DataFrame.from_dict(total, orient='index', columns=["uses"])
total_df.reset_index(level=0, inplace=True)
total_df.head()

,index,uses
0,None assigned,45518
1,gpu0001,857
2,gpu0002,571
3,gpu0003,770
4,gpu0004,628


In [135]:
overall_df = pd.merge(total_df, failed_df, how='outer')
overall_df.head()

,index,uses,failures
0,None assigned,45518,45.0
1,gpu0001,857,141.0
2,gpu0002,571,142.0
3,gpu0003,770,100.0
4,gpu0004,628,105.0


In [136]:
overall_df.shape

(574, 3)

In [137]:
total_df.shape

(574, 2)

In [138]:
overall_df['Pct_Fail'] = ""
overall_df.head()

,index,uses,failures,Pct_Fail
0,None assigned,45518,45.0,
1,gpu0001,857,141.0,
2,gpu0002,571,142.0,
3,gpu0003,770,100.0,
4,gpu0004,628,105.0,


In [139]:
overall_df['failures'].fillna(0, inplace=True)

In [140]:
overall_df['Pct_Fail']=overall_df.failures/overall_df.uses*100
overall_df.head()

,index,uses,failures,Pct_Fail
0,None assigned,45518,45.0,0.098862
1,gpu0001,857,141.0,16.452742
2,gpu0002,571,142.0,24.868651
3,gpu0003,770,100.0,12.987013
4,gpu0004,628,105.0,16.719745


In [141]:
overall_df.sort_values("Pct_Fail", ascending = False)

,index,uses,failures,Pct_Fail
503,vmp465,10,9.0,90.000000
468,vmp430,8,7.0,87.500000
299,vmp1370,3350,1944.0,58.029851
538,vmp588,64,37.0,57.812500
304,vmp1375,2833,1549.0,54.677021
302,vmp1373,2350,1245.0,52.978723
301,vmp1372,2513,1323.0,52.646240
300,vmp1371,1798,816.0,45.383760
5,gpu0005,447,168.0,37.583893
464,vmp425,907,332.0,36.604190


ELIMINATE ERROR CODES FOR USER ERROR